### Step 1: Combine Flow Cell Data

Concatenate FASTQ files from both flow cells into a single file. Check basic statistics like total read count
and file size to confirm successful combination.

In [3]:
!cd reads && sh sample_reads.sh

Starting subsampling at Wed Nov 19 09:45:18 CST 2025
pigz: abort: missing parameter after -p
pigz: abort: missing parameter after -p
cat: write error: Broken pipe
Finished subsampling at Wed Nov 19 09:45:18 CST 2025
Output file size:
-rw-r--r-- 1 willetse0745 SFU_Users 0 Oct  9 17:24 planaria_test_subset.fastq.gz
Number of reads in output:

gzip: planaria_test_subset.fastq.gz: unexpected end of file
0


Writes 1 million subsampled reads to `/data/groups/wheelenj/sequencing/20250916_M009242/planaria_test_subset.fastq.gz`

### Step 2: Initial Quality Assessment

Run NanoPlot on the combined dataset to assess read length distribution, quality scores, and overall data
characteristics. Optionally check for adapter contamination using Porechop.

In [4]:
!mkdir fastq_planaria
!mkdir fastq_planaria/qc
!NanoPlot --fastq /data/groups/wheelenj/sequencing/20250916_M009242/planaria_test_subset.fastq.gz -o fastq_planaria/qc

mkdir: cannot create directory ‘fastq_planaria’: File exists
mkdir: cannot create directory ‘fastq_planaria/qc’: File exists


In [ ]:
!multiqc fastq_planaria/.

### Step 3: Read Filtering and Subsampling

Use Filtlong to filter reads based on quality and length, targeting 100-150 GB of high-quality data
(approximately 50-100x coverage). Filter parameters: minimum length 1000 bp, minimum mean quality 8,
keep best 20-25% of reads.

In [ ]:
!mkdir trimmed_fastq
!filtlong --min_length 1000 --keep_percent 80 --min_mean_q 8 /data/groups/wheelenj/sequencing/20250916_M009242/planaria_test_subset.fastq.gz | gzip > trimmed_fastq/trimmed_reads.fastq.gz

mkdir: cannot create directory ‘trimmed_fastq’: File exists

Scoring long reads
  1,000,000 reads (6,122,479,215 bp) reads (62,569,487 bp)11,495 reads (69,480,054 bp)51,114 reads (308,517,951 bp)52,252 reads (315,391,128 bp)322,258,455 bp)75,879 reads (459,163,051 bp)521,235,318 bp)94,178 reads (570,513,172 bp)624,922,251 bp)108,094 reads (656,056,755 bp)666,416,747 bp)110,796 reads (673,296,661 bp)116,101 reads (706,869,259 bp)117,717 reads (717,200,446 bp)118,928 reads (724,082,766 bp)121,171 reads (737,864,220 bp)908,929,618 bp)168,259 reads (1,030,810,015 bp)173,154 reads (1,059,848,455 bp)1,073,679,473 bp)177,616 reads (1,087,521,831 bp)180,592 reads (1,104,816,013 bp)1,150,592,600 bp)1,254,918,420 bp)224,669 reads (1,377,294,317 bp)231,896 reads (1,421,244,554 bp)245,852 reads (1,508,258,156 bp)1,777,781,027 bp)320,527 reads (1,966,858,119 bp)2,023,909,420 bp)330,538 reads (2,027,377,019 bp)338,932 reads (2,079,153,747 bp)2,141,098,718 bp)349,629 reads (2,144,544,373 bp)358,433 r

### Step 4: Post-Filtering Quality Control

Run NanoPlot on the combined dataset to assess read length distribution, quality scores, and overall data
characteristics. Optionally check for adapter contamination using Porechop.

In [ ]:
!NanoPlot --fastq trimmed_fastq/trimmed_reads.fastq.gz -o trimmed_fastq/qc --plots kde hex

This command requires Kaleido v1.0.0 or greater.
Install it using `pip install 'kaleido>=1.0.0'` or `pip install 'plotly[kaleido]'`."



In [ ]:
!multiqc fastq_planaria/. trimmed_fastq/.


/// ]8;id=815645;https://multiqc.info\MultiQC]8;;\ 🎃 v1.31

     version_check | MultiQC Version v1.32 now available!
       file_search | Search path: /data/users/willetse0745/Planaria-Genome-Project/fastq_planaria
       file_search | Search path: /data/users/willetse0745/Planaria-Genome-Project/trimmed_fastq
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 34/34  astq_planaria/qc/NanoPlot-report.html
          nanostat | Found 1 reports
     write_results | Existing reports found, adding suffix to filenames. Use '--force' to overwrite.
     write_results | Data        : multiqc_data_1_2
     write_results | Report      : multiqc_report_2.html
           multiqc | MultiQC complete


### Step 5: Flye Assembly

Run Flye genome assembler using filtered nanopore reads. Use 48 CPU cores, 200GB RAM, allow 3 days
runtime. Flye handles raw nanopore reads well and excels at repeat resolution.

In [ ]:
!mkdir test_assembly
!mkdir test_assembly/flye_output

mkdir: cannot create directory ‘test_assembly’: File exists


In [ ]:
!flye --nano-raw trimmed_fastq/trimmed_reads.fastq.gz \
     --out-dir /data/groups/wheelenj/sequencing/20250916_M009242/test_assembly/flye_output/ \
     --threads 64 \
     --genome-size 1g \
     --iterations 2

[2025-10-30 21:53:54] INFO: Starting Flye 2.9.6-b1802
[2025-10-30 21:53:54] INFO: >>>STAGE: configure
[2025-10-30 21:53:54] INFO: Configuring run
[2025-10-30 21:55:05] INFO: Total read length: 4897987557
[2025-10-30 21:55:05] INFO: Input genome size: 1000000000
[2025-10-30 21:55:05] INFO: Estimated coverage: 4
[2025-10-30 21:55:05] WARNING: Expected read coverage is 4, the assembly is not guaranteed to be optimal in this setting. Are you sure that the genome size was entered correctly?
[2025-10-30 21:55:05] INFO: Reads N50/N90: 16392 / 6390
[2025-10-30 21:55:05] INFO: Minimum overlap set to 6000
[2025-10-30 21:55:05] INFO: >>>STAGE: assembly
[2025-10-30 21:55:05] INFO: Assembling disjointigs
[2025-10-30 21:55:05] INFO: Reading sequences
[2025-10-30 21:56:08] INFO: Counting k-mers:
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2025-10-30 21:58:08] INFO: Filling index table (1/2)
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2025-10-30 21:58:56] INFO: Filling index table (2/2)
0% 10% 20%

### Step 7: Initial Assembly Assessment

Run BUSCO analysis on both assemblies using metazoa_odb10 lineage to assess gene completeness.
Calculate basic assembly statistics with QUAST.

In [ ]:
!busco -i /data/groups/wheelenj/sequencing/20250916_M009242/test_assembly/flye_output/assembly.fasta \
      -l metazoa_odb10 \
      -o busco_test \
      -m genome \
      --cpu 64 \
      --out_path /data/groups/wheelenj/sequencing/20250916_M009242/

2025-11-12 11:29:08 INFO:	***** Start a BUSCO v6.0.0 analysis, current time: 11/12/2025 11:29:08 *****
2025-11-12 11:29:08 INFO:	Configuring BUSCO with local environment
2025-11-12 11:29:08 INFO:	Running genome mode
2025-11-12 11:29:08 INFO:	Downloading information on latest versions of BUSCO data...
2025-11-12 11:29:10 INFO:	Input file is /data/groups/wheelenj/sequencing/20250916_M009242/test_assembly/flye_output/assembly.fasta
2025-11-12 11:29:10 INFO:	Downloading file 'https://busco-data.ezlab.org/v5/data/lineages/metazoa_odb10.2024-01-08.tar.gz'
2025-11-12 11:29:15 INFO:	Decompressing file '/data/users/willetse0745/Planaria-Genome-Project/busco_downloads/lineages/metazoa_odb10.tar.gz'
2025-11-12 11:29:22 INFO:	Running BUSCO using lineage dataset metazoa_odb10 (eukaryota, 2024-01-08)
2025-11-12 11:29:22 INFO:	Running 1 job(s) on bbtools, starting at 11/12/2025 11:29:22
2025-11-12 11:29:24 INFO:	Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limi

## Subsample BUSCO Results

- Complete genes: 28.3% -- 270 total -> 91 contain internal stop codons
- Single copy: 25.7% -- 245
- Duplicated: 2.6% -- 25
- Fragmented genes: 8.0% -- 76
- Missing genes: 63.7% -- 608

In [ ]:
!multiqc fastq_planaria/. trimmed_fastq/qc/. /data/groups/wheelenj/sequencing/20250916_M009242/busco_test/short_summary.specific.metazoa_odb10.busco_test.txt --force --verbose   --dirs \
  --dirs-depth 1 \
  --force

[2025-11-12 12:18:24] multiqc.core.tmp_dir                               [DEBUG  ]  Using new temporary directory: /local/scratch/job_334086/tmpkd48xyif
[2025-11-12 12:18:24] root                                               [DEBUG  ]  Logging to file: /local/scratch/job_334086/tmpkd48xyif/multiqc.log

/// ]8;id=619933;https://multiqc.info\MultiQC]8;;\ 🔍 v1.31

[2025-11-12 12:18:25] multiqc.core.update_config                         [DEBUG  ]  This is MultiQC v1.31
[2025-11-12 12:18:25] multiqc.core.update_config                         [DEBUG  ]  Running Python 3.12.12 | packaged by conda-forge | (main, Oct 13 2025, 14:34:15) [GCC 14.3.0]
[2025-11-12 12:18:25] multiqc.core.update_config                         [INFO   ]  Prepending directory to sample names
[2025-11-12 12:18:25] urllib3.connectionpool                             [DEBUG  ]  Starting new HTTPS connection (1): api.multiqc.info:443
[2025-11-12 12:18:25] urllib3.connectionpool                             [DEBUG  ]  ht

## Phase 3: Assembly Polishing

### Step 8: First Round Racon Polishing

Run Racon polishing using original filtered reads. This
provides additional consensus improvement through multiple alignment iterations.

In [1]:
!mkdir minimap2_subsample
!minimap2 -ax map-ont -t 16 /data/groups/wheelenj/sequencing/20250916_M009242/test_assembly/flye_output/assembly.fasta trimmed_fastq/trimmed_reads.fastq.gz > minimap2_subsample/aligned.sam

mkdir: cannot create directory ‘minimap2_subsample’: File exists
[M::mm_idx_gen::6.779*1.40] collected minimizers
[M::mm_idx_gen::7.230*2.20] sorted minimizers
[M::main::7.232*2.20] loaded/built the index for 15138 target sequence(s)
[M::mm_mapopt_update::7.684*2.13] mid_occ = 524
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 15138
[M::mm_idx_stat::8.007*2.08] distinct minimizers: 25863390 (63.74% are singletons); average occurrences: 3.222; average spacing: 5.289; total length: 440718210
[M::worker_pipeline::176.722*14.74] mapped 37960 sequences
[M::worker_pipeline::335.830*15.33] mapped 37806 sequences
[M::worker_pipeline::495.245*15.53] mapped 37895 sequences
[M::worker_pipeline::654.713*15.63] mapped 38194 sequences
[M::worker_pipeline::814.324*15.69] mapped 38208 sequences
[M::worker_pipeline::971.814*15.74] mapped 37845 sequences
[M::worker_pipeline::1132.097*15.75] mapped 38052 sequences
[M::worker_pipeline::1290.742*15.77] mapped 38032 sequences
[M::worker_pipeline

In [1]:
!mkdir racon_subsample
!./racon/build/bin/racon -t 16 trimmed_fastq/trimmed_reads.fastq.gz minimap2_subsample/aligned.sam /data/groups/wheelenj/sequencing/20250916_M009242/test_assembly/flye_output/assembly.fasta > racon_subsample/racon_polished1.fasta

[racon::Polisher::initialize] loaded target sequences 5.372676 s
[racon::Polisher::initialize] loaded sequences 158.373712 s
[racon::Polisher::initialize] loaded overlaps 145.828169 s
[racon::Polisher::initialize] aligning overlaps [====================] 6.169021 s
[racon::Polisher::initialize] transformed data into windows 18.140456 s
[racon::Polisher::polish] generating consensus [====================] 1464.840097 s
[racon::Polisher::] total = 1799.908450 s


### Step 9: Second Round Racon Polishing

Run Racon polishing using original filtered reads. This
provides additional consensus improvement through multiple alignment iterations.

In [2]:
!minimap2 -ax map-ont -t 16 racon_subsample/racon_polished1.fasta trimmed_fastq/trimmed_reads.fastq.gz > minimap2_subsample/aligned2.sam

[M::mm_idx_gen::6.731*1.40] collected minimizers
[M::mm_idx_gen::7.219*2.22] sorted minimizers
[M::main::7.220*2.22] loaded/built the index for 14520 target sequence(s)
[M::mm_mapopt_update::7.678*2.15] mid_occ = 514
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 14520
[M::mm_idx_stat::8.002*2.10] distinct minimizers: 25867670 (63.57% are singletons); average occurrences: 3.229; average spacing: 5.281; total length: 441135934
[M::worker_pipeline::177.776*14.75] mapped 37960 sequences
[M::worker_pipeline::338.709*15.34] mapped 37806 sequences
[M::worker_pipeline::499.722*15.54] mapped 37895 sequences
[M::worker_pipeline::661.124*15.64] mapped 38194 sequences
[M::worker_pipeline::822.614*15.70] mapped 38208 sequences
[M::worker_pipeline::982.188*15.75] mapped 37845 sequences
[M::worker_pipeline::1144.661*15.75] mapped 38052 sequences
[M::worker_pipeline::1304.980*15.78] mapped 38032 sequences
[M::worker_pipeline::1466.235*15.80] mapped 37806 sequences
[M::worker_pipeline::159

In [5]:
!./racon/build/bin/racon -t 16 trimmed_fastq/trimmed_reads.fastq.gz minimap2_subsample/aligned2.sam racon_subsample/racon_polished1.fasta > racon_subsample/racon_polished2.fasta

[racon::Polisher::initialize] loaded target sequences 4.979467 s
[racon::Polisher::initialize] loaded sequences 155.998131 s
[racon::Polisher::initialize] loaded overlaps 139.839154 s
[racon::Polisher::initialize] aligning overlaps [====================] 6.084883 s
[racon::Polisher::initialize] transformed data into windows 18.249343 s
[racon::Window::generate_consensus] warning: contig 9427 might be chimeric in window 10!
[racon::Window::generate_consensus] warning: contig 10282 might be chimeric in window 47!
[racon::Polisher::polish] generating consensus [====================] 1466.331903 s
[racon::Polisher::] total = 1792.615138 s


### Step 10: First Round Medaka Polishing

Use Medaka for nanopore-specific error correction on the selected assembly. This corrects systematic
nanopore sequencing errors and improves consensus accuracy.

In [7]:
!medaka_consensus -i trimmed_fastq/trimmed_reads.fastq.gz \
                 -d racon_subsample/racon_polished2.fasta\
                 -o test_assembly/medaka_polish \
                 -t 16

Attempting to automatically select model version.
SUCCESS: Automatic model selection chose model: '/data/users/willetse0745/.conda/envs/planaria-env/lib/python3.12/site-packages/medaka/data/r1041_e82_400bps_sup_v4.3.0_model_pt.tar.gz'
Checking program versions
This is medaka 2.1.1
Program    Version    Required   Pass     
bcftools   1.22       1.11       True     
bgzip      1.22.1     1.11       True     
minimap2   2.30       2.11       True     
samtools   1.22.1     1.11       True     
tabix      1.22.1     1.11       True     
[12:57:52 - MdlStrTGZ] Successfully removed temporary files from /local/scratch/job_336804/tmpb37ribk7.
[12:57:53 - MdlStrTGZ] Successfully removed temporary files from /local/scratch/job_336804/tmph5sg7k4q.
[12:57:55 - MdlStrTGZ] Successfully removed temporary files from /local/scratch/job_336804/tmp7uaxnfj2.
[12:57:55 - MdlStrTGZ] Successfully removed temporary files from /local/scratch/job_336804/tmpzy3kap4h.
Aligning basecalls to draft
Creating fai ind

### Step 11: Post-polishing Quality Assessment

Run BUSCO again on polished assembly to ensure polishing improved rather than degraded gene
completeness. Compare metrics with initial assembly.

In [3]:
!busco -i test_assembly/medaka_polish/consensus.fasta \
      -l metazoa_odb10 \
      -o busco_test_medaka_racon \
      -m genome \
      --cpu 64 \
      --force \
      --out_path /data/groups/wheelenj/sequencing/20250916_M009242/

2025-11-14 13:29:28 INFO:	***** Start a BUSCO v6.0.0 analysis, current time: 11/14/2025 13:29:28 *****
2025-11-14 13:29:28 INFO:	Configuring BUSCO with local environment
2025-11-14 13:29:28 INFO:	Running genome mode
2025-11-14 13:29:28 INFO:	'Force' option selected; overwriting previous results directory
2025-11-14 13:29:28 INFO:	Downloading information on latest versions of BUSCO data...
2025-11-14 13:29:30 INFO:	Input file is /data/users/willetse0745/Planaria-Genome-Project/test_assembly/medaka_polish/consensus.fasta
2025-11-14 13:29:30 INFO:	The local file or folder /data/users/willetse0745/Planaria-Genome-Project/busco_downloads/lineages/metazoa_odb10 is the last available version.
2025-11-14 13:29:33 INFO:	Running BUSCO using lineage dataset metazoa_odb10 (eukaryota, 2024-01-08)
2025-11-14 13:29:33 INFO:	Running 1 job(s) on bbtools, starting at 11/14/2025 13:29:33
2025-11-14 13:29:34 INFO:	Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit o

## Subsample BUSCO Results Post Medaka Polishing ONLY

- Complete genes: 28.4% -- 271 total -> 93 contain internal stop codons
- Single copy: 25.8% -- 246
- Duplicated: 2.6% -- 25
- Fragmented genes: 7.5% -- 72
- Missing genes: 64.0% -- 611

## Subsample BUSCO Results Post Racon and Medaka Polishing

- Complete genes: 
- Single copy: 
- Duplicated: 
- Fragmented genes:
- Missing genes: